### This notebook scrapes schools level data from schools.org.in

### Import required packages

In [10]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from inscriptis import get_text

### Function to retrieve the statewise number of schools and links to data on all schools in the state

In [2]:
def get_table_contents3(url):
    
    # open the link to the webpage
    # browser.get(url)

    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    table_container = soup.find_all('table')
    soup = BeautifulSoup(str(table_container), 'html.parser')

    i = 1
    j = 1
    dic = {}
    
    for each in soup.find_all('td'):

        if(i%3==2):
            temp_state = each.text[each.text.find('Schools in')+11:]
            temp_link = 'https://schools.org.in/'+each.a.get('href')

        elif(i%3==0):
            temp_count = each.text
            dic[j] = {'state':temp_state, 'link': temp_link, 'number of schools':temp_count}
            j+=1

        i+=1

    # save the dictionary into a dataframe
    df_state=pd.DataFrame.from_dict(dic).T
    
    return df_state

### Function to retrieve links to all the districts within each state (to get school data within those districts)

In [3]:
def get_table_contents2(url):
    
    # open the link to the webpage
    #browser.get(url)

    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    table_container = soup.find_all('table')
    soup = BeautifulSoup(str(table_container), 'html.parser')

    i = 1
    j = 1
    dic = {}

    for each in soup.find_all('td'):
        if(i%2==0):
            temp_state = each.text[each.text.find('Schools in')+11:]
            temp_link = 'https://schools.org.in/'+each.a.get('href')
            dic[j] = {'col1':temp_state, 'col2': temp_link}
            j+=1

        i+=1

    # save the dictionary into a dataframe
    df_state=pd.DataFrame.from_dict(dic).T
    
    return df_state

### Function to retrive data on all schools within a district

In [4]:
def get_table_contents_school(url):
    
    # open the link to the webpage
    #browser.get(url)

    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    table_container = soup.find_all('table')
    soup = BeautifulSoup(str(table_container), 'html.parser')

    i = 1
    j = 1
    dic = {}

    for each in soup.find_all('td'):

        if(i%2==0):
            temp_state = each.text
            temp_link = 'https://schools.org.in/'+each.a.get('href')
            dic[j] = {'col1':temp_state, 'col2': temp_link}
            j+=1

        i+=1

    # save the dictionary into a dataframe
    df_state=pd.DataFrame.from_dict(dic).T
    return df_state

### Retrive school level data from school links

In [5]:
def get_school_data(url):

    vars = ['Instruction Medium', 'Male Teachers', 'Pre Primary Sectin Avilable', 'Board for Class 10<', 'School Type', 'Classes','Female Teacher', 'Pre Primary Teachers','Board for Class 10+2', 'Meal',  'Establishment', 'School Area', 'School Shifted to New Place', 'Head Teachers', 'Head Teacher:', 'Is School Residential', 'Residential Type', 'Total Teachers', 'Contract Teachers', 'Management', 'Village / Town', 'Cluster', 'Block','District','State','UDISE Code','Building','Class Rooms','Boys Toilet', 'Girls Toilet', 'Computer Aided Learning','Electricity', 'Wall','Library', 'Playground','Books in Library','Drinking Water','Ramps for Disable','Computers','Pincode']
    # open the link to the webpage
    #browser.get(url)

    lis = []
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    table_container = soup.find_all('li', class_='list-group-item')

    for each in table_container:

        temp_str = str(each)
        temp_header = temp_str[temp_str.find('</i>')+4:][:temp_str[temp_str.find('</i>')+4:].find('<b')]
        temp_value = temp_str[temp_str.find('<b>')+3:][:temp_str[temp_str.find('<b>')+3:].find('</b')]

        if (temp_header==' '):
            temp_header = temp_str[temp_str.find('<b>')+3:][:temp_str[temp_str.find('<b>')+3:].find('</b>')]
            temp_value = temp_str[temp_str.find('</b>')+4:][:temp_str[temp_str.find('</b>')+4:].find('</li')]

        for item in vars:
            if (item in temp_header):
                if (not((item=='Library') & ('Books in Library' in temp_header))):
                    lis.append(temp_value)

    txt = get_text(html.text)
    lis.append(txt[txt.find('PIN Code')+11:txt.find('PIN Code')+17])
    full_d = dict(zip(vars,lis))
    
    return full_d

### Use the above functions to scrape data

In [ ]:
url = "https://schools.org.in/schools-in-india.html"

school={}
j=0
flag = 0

state_school = get_table_contents3(url)
print(state_school)
for each in state_school['link']:
    print(each)
    df1 = get_table_contents2(each)
    for item in df1['col2']:

        df2 = get_table_contents2(item)
        for link in df2['col2']:

            df3 = get_table_contents2(link)
            for piece in df3['col2']:
                
                df4 = get_table_contents_school(piece)
                if not df4.empty:
                    
                    for this in df4['col2']:
                        temp_state = state_school[state_school['link'] == each ]['state'].item()
                        temp_district = df1[df1['col2']==item]['col1'].item()
                        temp_block = df2[df2['col2']==link]['col1'].item()
                        temp_cluster = df3[df3['col2']==piece]['col1'].item()
                        temp_school = df4[df4['col2']==this]['col1'].item()
                        temp_dict={'state':temp_state, 'district': temp_district ,'block': temp_block, 'cluster': temp_cluster, 'school': temp_school}
                        temp_dict.update(get_school_data(this))
                        school[j]=temp_dict
                        j+=1

In [12]:
# save the dictionary into a dataframe
df_data=pd.DataFrame.from_dict(school).T
# save the dataframe into an excel file
df_data.to_excel('school_detail_data.xlsx',engine='xlsxwriter')